# Automação Web: Buscador de Preços

## Entendendo a Situação

Como Analista de Compras sua principal tarefa consiste em identificar qual fornecedor oferece o melhor preço para cada um dos produtos de sua empresa.

Diariamente, você busca no Google Shopping e no Buscapé os preços dos seguintes produtos: iPhone 12 64GB e Placa de Vídeo RTX 3060.

A fim de tornar sua busca mais refinada, você usa os critérios definidos na planilha `produtos.xlsx`, a qual especifica os preços mínimo e máximo para cada produto bem como os termos banidos da busca.

Por fim, você sumariza os resultados de sua busca numa tabela que é enviada por e-mail para o seguinte endereço: diegotorrescoder@gmail.com.

## Divisão do Problema

Para automatizar este processo, vamos adotar o seguinte passo a passo:

- Criar um navegador
- Importar a base de dados
- Visualizar a base de dados
- Procurar cada produto no Google Shopping
    - Verificar se o produto do anúncio possui todos os termos da busca
    - Verificar se o produto do anúncio não contém os termos banidos
    - Verificar se o produto do anúncio está dentro da faixa de preços admitida
- Procurar cada produto no Buscapé
- Procurar cada produto no Google Shopping
    - Verificar se o produto do anúncio possui todos os termos da busca
    - Verificar se o produto do anúncio não contém os termos banidos
    - Verificar se o produto do anúncio está dentro da faixa de preços admitida
- Salvar as ofertas em um dataframe
- Exportar o dataframe como uma planilha do Excel
- Enviar por e-mail os resultados    

### Importações

In [1]:
# Importa o webdriver a partir da biblioteca selenium
from selenium import webdriver

# Importa a classe By
from selenium.webdriver.common.by import By

# Importa a classe Keys
from selenium.webdriver.common.keys import Keys

# Importa o gerenciador de webdriver do Google Chrome
from webdriver_manager.chrome import ChromeDriverManager

# Importa a classe Service
from selenium.webdriver.chrome.service import Service

# Importa a função WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait

# Importa a função expected_conditions
from selenium.webdriver.support import expected_conditions as EC

# Importa o pandas com o apelido pd
import pandas as pd

### Criar um Navegador

In [2]:
# Faz o download do webdriver do Google Chrome
servico = Service(ChromeDriverManager().install())

# Cria um navegador do Google Chrome
navegador = webdriver.Chrome(service=servico)

### Importar e Visualizar a Base de Dados

In [3]:
# Lê a base de dados em Excel
df_produtos = pd.read_excel('buscas.xlsx')

# Exibe a base de dados
display(df_produtos)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch usado seminovo,3000,3500
1,rtx 3060,zota galax,4000,4500


### Funções Auxiliares

A função a seguir retorna `True` caso o nome do produto anunciado possua pelo menos um dos termos banidos:

In [4]:
def possui_termos_banidos(lista_termos_banidos, titulo_anuncio):
    # Variável auxiliar inicializada com False
    possui_termos_banidos = False

    # Percorre os elementos da lista de termos banidos
    for termo in lista_termos_banidos:
        # Verifica se o nome do produto anunciado contém pelo menos um termo banido
        if termo in titulo_anuncio:
            # Atualiza o valor da variável auxiliar
            possui_termos_banidos = True
            
    return possui_termos_banidos

A função a seguir retorna `True` caso o nome do produto anunciado possua todos os termos da busca:

In [5]:
def possui_todos_termos(lista_termos_busca, titulo_anuncio):
    # Variável auxiliar inicializada com True
    possui_todos_termos_busca = True

    # Percorre a lista de termos da busca
    for termo in lista_termos_busca:
        # Verifica se o termo desta iteração não está contido no nome
        if termo not in titulo_anuncio:
            # Atualiza a variável auxiliar
            possui_todos_termos_busca = False
            
    return possui_todos_termos_busca

### Procurar o Produto no Google Shopping

Esta função faz a busca de ofertas no **Google Shopping**:

In [ ]:
def buscar_ofertas_google_shopping(navegador, produto, termos_banidos, preco_min, preco_max):
    '''Faz a busca por ofertas no Google Shopping.'''
    
    # Coloca todas as letras do nome do produto em minúsculas
    nome_produto = produto.lower()

    # Lista com as palavras do nome do produto
    lista_termos_busca = nome_produto.split(' ')

    # Coloca todas as letras dos termos de busca em minúsculas
    termos_banidos = termos_banidos.lower()

    # Lista de termos banidos
    lista_termos_banidos = termos_banidos.split(' ')
    
    # Preço mínimo
    preco_min = float(preco_min)

    # Preço máximo
    preco_max = float(preco_max)
    
    # Lista com as ofertas encontradas (lista de tuplas)
    ofertas_encontradas = []
    
    # Acessar a página do Google
    navegador.get('https://www.google.com.br/')

    # Localiza o campo de buscas do Google
    campo_busca = navegador.find_element(
        By.XPATH, 
        '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input'
    )

    # Digitar nome do produto no campo de buscas
    campo_busca.send_keys(nome_produto)

    # Dá enter para buscar
    campo_busca.send_keys(Keys.RETURN)

    # Seleciona todos os elementos da barra de links
    links = navegador.find_elements(By.CLASS_NAME, 'hdtb-mitem')

    # Percorre a lista de links da barra de navegação
    for link in links:
        # Verifica se o texto especificado está dentro do item
        if 'Shopping' in link.text:
            # Clica no link
            link.click()

            # Interrompe o laço de repetição
            break

    # Obter as informações do produto
    # Seleciona todos os anúncios que possuem a classe especificada
    anuncios = navegador.find_elements(By.CLASS_NAME, 'i0X6df')

    # Percorre a lista de anúncios
    for anuncio in anuncios:
        # Obtém o nome do produto
        nome_anuncio = anuncio.find_element(By.CLASS_NAME, 'tAxDx').text

        # Coloca todas as letras do nome do produto em minúsculas
        nome_anuncio = nome_anuncio.lower()
        
        # Chama a função para verificar se o nome do produto contém algum termo banido
        tem_termos_banidos = possui_termos_banidos(lista_termos_banidos, nome_anuncio)

        # Chama a função para verificar se o nome do produto contém todos os termos da busca
        tem_todos_termos_busca = possui_todos_termos()

        # Verifica se o produto do anúncio atende a estas condições:
        # 1. O nome do produto no anúncio NÃO possui qualquer dos termos banidos
        # 2. O nome do produto possui todos os termos da busca
        if not tem_termos_banidos and tem_todos_termos_busca:
            # Obtém o preço do produto
            preco_anuncio = anuncio.find_element(By.CLASS_NAME, 'a8Pemb').text

            # Trata a string com o preço do produto
            # 1. Remove o R$
            # 2. Remove espaços em branco
            # 3. Remove os pontos
            # 4. Troca a vírgula pelo ponto
            preco_anuncio = preco_anuncio.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')

            # Converte a string num float
            preco_anuncio = float(preco_anuncio)

            # Verifica se o preço no anúncio está no intervalo de preço mínimo e máximo
            if preco_anuncio >= preco_min and preco_anuncio <= preco_max:
                # Elemento de referência (filho) para localizar o link
                elemento_filho = anuncio.find_element(By.CLASS_NAME, 'KoNVE')

                # Elemento que contém o link
                elemento_pai = elemento_filho.find_element(By.XPATH, '..')

                # Link do produto
                link_anuncio = elemento_pai.get_attribute('href')

                # Acrescenta uma tupla à lista de ofertas encontradas
                ofertas_encontradas.append((nome_anuncio, preco_anuncio, link_anuncio))

    # Retorna a lista de ofertas
    return ofertas_encontradas

### Procurar o Produto no Buscapé

A função a seguir servirá para buscar informações no **Buscapé**:

In [13]:
def buscar_ofertas_buscape(navegador, produto, termos_banidos, preco_min, preco_max ):
    '''Faz uma busca por ofertas no Buscapé'''    
    # Coloca todas as letras do nome do produto em minúsculas
    nome_produto = produto.lower()
    
    # Obtém todos os termos da busca, dividindo o texto da busca com base nos espaços
    lista_termos_busca = nome_produto.split(' ')
    
    # Coloca todas as letras dos termos banidos em minúsculas
    termos_banidos = termos_banidos.lower()
    
    # Obtém todos os termos banidos
    lista_termos_banidos = termos_banidos.split(' ')
    
    # Preço mínimo
    preco_min = float(preco_min)

    # Preço máximo
    preco_max = float(preco_max)
    
    # Lista de ofertas encontradas
    ofertas_encontradas = []

    # Acessar o site do Buscapé para fazer uma busca
    navegador.get('https://www.buscape.com.br/')
    
    # Localiza o campo de busca pelo XPATH
    campo_busca = navegador.find_element(
        By.XPATH, 
        '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input'
    )
    
    # Digita a busca no campo de buscas e faz a pesquisa
    campo_busca.send_keys(nome_produto, Keys.RETURN)

    # Aguarda no máximo 30s para que o elemento seja localizado
    elemento = WebDriverWait(navegador, 30).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'SearchHeader_titleSection__5SEk9'))
    )

    # Localiza todos os anúncios
    anuncios = navegador.find_elements(By.CLASS_NAME, 'SearchCard_ProductCard_Inner__7JhKb')
    
    # Percorre a lista de anúncios
    for anuncio in anuncios:
        # Obtém o nome do produto
        nome_anuncio = anuncio.find_element(By.CLASS_NAME, 'SearchCard_ProductCard_Name__ZaO5o').text

        # Coloca as palavras do nome do produto em minúsculas
        nome_anuncio = nome_produto.lower()
        print(nome_anuncio)
        
        # Chama a função para verificar se o nome do produto possui termos banidos
        tem_termos_banidos = possui_termos_banidos(lista_termos_banidos, nome_anuncio)

        # Chama a função para verificar se o nome do produto possui todos os termos da busca
        tem_todos_termos_busca = possui_todos_termos(lista_termos_busca, nome_anuncio)

        if not tem_termos_banidos and tem_todos_termos_busca:
            # Obtém o preço do produto
            preco_anuncio = anuncio.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__Zxam2').text

            # Trata a string com o preço do produto
            # 1. Remove o R$
            # 2. Remove espaços em branco
            # 3. Remove os pontos
            # 4. Troca a vírgula pelo ponto
            preco_anuncio = preco_anuncio.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')

            # Converte a string num float
            preco_anuncio = float(preco_anuncio)
            
            # Verifica se o preço no anúncio está no intervalo de preço mínimo e máximo
            if preco_anuncio >= preco_min and preco_anuncio <= preco_max:               
                # Obtém o atributo href do link
                link_anuncio = anuncio.get_attribute('href')

                # Acrescenta uma tupla à lista de ofertas
                ofertas_encontradas.append((nome_anuncio, preco_anuncio, link_anuncio))
                
    # Retorna a lista de ofertas encontradas
    return ofertas_encontradas

A partir deste ponto, vamos obter as ofertas no Google Shopping e armazená-las num dataframe:

In [14]:
# Nome do produto
nome_produto = 'iphone 12 64gb'

# Termos banidos
termos_banidos = 'mini watch usado seminovo'

# Preço mínimo
preco_min = 3000

# Preço máximo
preco_max = 3800

# Exibe as ofertas encontradas
# ofertas = buscar_ofertas_google_shopping(navegador, nome_produto, termos_banidos, preco_min, preco_max)
ofertas = buscar_ofertas_buscape(navegador, nome_produto, termos_banidos, preco_min, preco_max)

# Cria um dataframe com as ofertas encontradas
df_ofertas = pd.DataFrame.from_records(ofertas, columns=['Produto', 'Preço (R$)', 'Link Oferta'])

# Exibe o dataframe
df_ofertas

iphone 12 64gb
iphone 12 64gb
iphone 12 64gb
iphone 12 64gb


,Produto,Preço (R$),Link Oferta
0,iphone 12 64gb,3781.00,https://www.buscape.com.br/celular/smartphone-...
1,iphone 12 64gb,3779.10,https://www.buscape.com.br/celular/smartphone-...
2,iphone 12 64gb,3193.52,https://www.buscape.com.br/celular/smartphone-...
